# Projekt WZUM predykcja all-NBA team i 
# all-rookie-team cz. 3

## Wczytanie danych

In [1]:
import pandas as pd 

In [2]:
all_nba_data = pd.read_csv("./csv/all_nba_cleaned.csv")
all_rookie_data = pd.read_csv("./csv/all_rookies_cleaned.csv")

In [3]:
all_nba_data = all_nba_data.drop(["Unnamed: 0"],axis=1)
all_rookie_data = all_rookie_data.drop(["Unnamed: 0"],axis=1)

In [4]:
all_nba_data = all_nba_data.dropna(subset=['Pos'])
# pd.isnull(all_nba_data)
# pd.isnull(all_nba_data).sum()
# pd.isnull(all_rookie_data).sum()
all_nba_data=all_nba_data.fillna(0)
all_rookie_data=all_rookie_data.fillna(0)

## Podzial danych do uczenia

In [5]:
all_nba_data.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'season',
       'Pts Won', 'Pts Max', 'Share'],
      dtype='object')

In [6]:
all_rookie_data.columns

Index(['Player', 'Debut', 'Age', 'Yrs', 'G', 'MP', 'FG', 'FGA', '3P', '3PA',
       'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1', 'TRB.1', 'AST.1', 'STL.1',
       'BLK.1', 'season', 'Pts Won', 'Pts Max', 'Share'],
      dtype='object')

In [7]:
p_all_nba = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'season']

p_all_rookie = [ 'Age', 'G', 'FG', 'FGA', '3P', '3PA',
       'FT', 'FTA', 'ORB', 'TOV', 'PF', 
       'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1', 'TRB.1', 'AST.1', 'STL.1',
       'BLK.1', 'season']
#  'GS', 'eFG%', 'DRB' 

In [8]:
# pd.isnull(all_nba_data).sum()

In [9]:
train_all_nba = all_nba_data[all_nba_data["season"] <2024]
test_all_nba = all_nba_data[all_nba_data["season"] == 2024]

train_all_rookie = all_rookie_data[all_rookie_data["season"] <2024]
test_all_rookie = all_rookie_data[all_rookie_data["season"] == 2024]

# Testowanie modeli

## **Test modelu regresji liniowej**

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
model1 = LinearRegression()
model1.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

model2 = LinearRegression()
model2.fit(train_all_rookie[p_all_rookie],train_all_rookie["Share"])

LinearRegression()

In [12]:
predykcja_all_nba = model1.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)

predykcja_rookie = model2.predict(test_all_rookie[p_all_rookie])
predykcja_rookie= pd.DataFrame(predykcja_rookie,columns =["Wynik rookie"],index=test_all_rookie.index)
wyniki_all_rookie = pd.concat([test_all_rookie[["Player","Share"]],predykcja_rookie ] ,axis=1).sort_values("Wynik rookie",ascending = False).head(15)



In [13]:
wyniki_all_nba
#Joel Embiid zagral za malo spotkan

,Player,Share,Wynik all nba
3686,Joel Embiid,0.000,0.529878
2562,Giannis Antetokounmpo,0.956,0.478567
4809,Luka Dončić,0.996,0.462661
6545,Shai Gilgeous-Alexander,1.000,0.415337
5635,Nikola Jokić,1.000,0.374432
399,Anthony Davis,0.465,0.312225
4702,LeBron James,0.331,0.306635
3432,Jayson Tatum,0.863,0.280627
7365,Victor Wembanyama,0.022,0.268117
4317,Kevin Durant,0.554,0.252338


In [14]:
from scipy.stats import spearmanr
spearman_corr, _ = spearmanr(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
spearman_corr


0.47736354530601505

In [15]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
mae

0.30781599226668316

In [16]:
from sklearn.metrics import ndcg_score
import numpy as np
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.7176833246374799

In [17]:
wyniki_all_rookie

,Player,Share,Wynik rookie
1298,Victor Wembanyama,1.000,1.453992
183,Chet Holmgren,1.000,1.139633
122,Brandon Miller,0.985,0.814428
793,Keyonte George,0.389,0.576197
1129,Scoot Henderson,0.167,0.519110
537,Jaime Jaquez Jr.,0.949,0.516485
43,Amen Thompson,0.505,0.414132
114,Brandin Podziemski,0.813,0.408643
420,GG Jackson II,0.217,0.394557
1235,Trayce Jackson-Davis,0.212,0.352540


In [18]:
spearman_corr, _ = spearmanr(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
spearman_corr

0.47736354530601505

In [19]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.7176833246374799

## Test modelu regresji liniowej z sprawdzeniem czy G>65 dla all-nba-team

In [20]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [21]:
class ReplaceLowGTransformer:
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X.loc[X['G'] < 65, ['eFG%','FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] = 0
        return X

# Create the pipeline
pipeline = Pipeline(steps=[
    ('replace_low_g', ReplaceLowGTransformer()),  # Custom transformer to replace values
    # ('scaler', StandardScaler()),  # Optional: Scale features
    ('regressor', LinearRegression())  # Linear Regression model
])


In [22]:
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
2562,Giannis Antetokounmpo,0.956,0.623340
4809,Luka Dončić,0.996,0.543832
6545,Shai Gilgeous-Alexander,1.000,0.534886
5635,Nikola Jokić,1.000,0.444444
4702,LeBron James,0.331,0.434068
399,Anthony Davis,0.465,0.431613
4317,Kevin Durant,0.554,0.418556
3432,Jayson Tatum,0.863,0.361974
7365,Victor Wembanyama,0.022,0.342599
1706,DeMar DeRozan,0.002,0.338491


In [23]:
spearman_corr, _ = spearmanr(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
spearman_corr

0.5622886534868442

In [24]:
mae = mean_absolute_error(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
mae

0.28456030934137944

In [25]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.908556440116767

## Drzewo decyzyjne 

In [26]:
from sklearn.tree import DecisionTreeRegressor
pipeline = Pipeline(steps=[
    ('replace_low_g', ReplaceLowGTransformer()),  # Custom transformer to replace values
    ('scaler', StandardScaler()),  # Optional: Sca1le features
    ('regressor', DecisionTreeRegressor(max_depth=15, min_samples_split=2))  # Linear Regression model
])
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
399,Anthony Davis,0.465,0.969000
4317,Kevin Durant,0.554,0.852000
2562,Giannis Antetokounmpo,0.956,0.806000
403,Anthony Edwards,0.576,0.806000
6545,Shai Gilgeous-Alexander,1.000,0.806000
4702,LeBron James,0.331,0.806000
5635,Nikola Jokić,1.000,0.806000
4809,Luka Dončić,0.996,0.806000
1419,Damian Lillard,0.000,0.782000
3432,Jayson Tatum,0.863,0.780000


In [27]:
spearman_corr, _ = spearmanr(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
spearman_corr

0.6623237131948324

In [28]:
mae = mean_absolute_error(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
mae

0.34514333333333325

In [29]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.8004368032221

## Random forest

In [30]:
from sklearn.ensemble import RandomForestRegressor
pipeline = Pipeline(steps=[
    ('replace_low_g', ReplaceLowGTransformer()),  # Custom transformer to replace values
    ('scaler', StandardScaler()),  # Optional: Sca1le features
    ('regressor', RandomForestRegressor(n_estimators=100, max_depth=10))  # Linear Regression model
])
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
2562,Giannis Antetokounmpo,0.956,0.865726
6545,Shai Gilgeous-Alexander,1.000,0.865492
5635,Nikola Jokić,1.000,0.858595
4809,Luka Dončić,0.996,0.856635
4702,LeBron James,0.331,0.825288
4317,Kevin Durant,0.554,0.740774
3432,Jayson Tatum,0.863,0.643169
403,Anthony Edwards,0.576,0.594535
399,Anthony Davis,0.465,0.550834
6698,Stephen Curry,0.236,0.545193


In [31]:
spearman_corr, _ = spearmanr(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
spearman_corr

0.8231755528197083

In [32]:
mae = mean_absolute_error(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
mae

0.21149919092981903

In [33]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.9636313152742895

In [34]:
from sklearn.svm import SVR
pipeline = Pipeline(steps=[
    ('replace_low_g', ReplaceLowGTransformer()),  # Custom transformer to replace values
    ('scaler', StandardScaler()),  # Optional: Sca1le features
    ('regressor', SVR(kernel='rbf', C=1.0, epsilon=0.1))  # Linear Regression model
])
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
4809,Luka Dončić,0.996,1.002432
2562,Giannis Antetokounmpo,0.956,0.866080
5635,Nikola Jokić,1.000,0.802325
4702,LeBron James,0.331,0.729386
6545,Shai Gilgeous-Alexander,1.000,0.724459
4317,Kevin Durant,0.554,0.559180
3432,Jayson Tatum,0.863,0.552237
6698,Stephen Curry,0.236,0.543984
399,Anthony Davis,0.465,0.503249
1988,Domantas Sabonis,0.210,0.401827


In [35]:
spearman_corr, _ = spearmanr(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
spearman_corr

0.791118306311477

In [36]:
mae = mean_absolute_error(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
mae

0.20437412906913818

In [37]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.9376605107461687

## Gradient boosting 


In [38]:
from sklearn.ensemble import GradientBoostingRegressor
pipeline = Pipeline(steps=[
    ('replace_low_g', ReplaceLowGTransformer()),  
    ('scaler', StandardScaler()),  
    ('regressor', GradientBoostingRegressor())  
])

In [39]:
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
4809,Luka Dončić,0.996,1.002232
6545,Shai Gilgeous-Alexander,1.000,0.986462
5635,Nikola Jokić,1.000,0.911038
4317,Kevin Durant,0.554,0.857966
2562,Giannis Antetokounmpo,0.956,0.831094
4702,LeBron James,0.331,0.789980
1892,Devin Booker,0.141,0.656092
403,Anthony Edwards,0.576,0.643860
3432,Jayson Tatum,0.863,0.620150
1645,De'Aaron Fox,0.018,0.541556


In [40]:
spearman_corr, _ = spearmanr(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
spearman_corr

0.766742128266255

In [41]:
mae = mean_absolute_error(wyniki_all_nba['Share'], wyniki_all_nba['Wynik all nba'])
mae

0.2157912804152236

In [42]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.9607854056441691

In [43]:
import pickle
with open('./pipeline/Model_all_nba.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

## Ridge


In [44]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=0.7)
pipeline = Pipeline(steps=[
    ('replace_low_g', ReplaceLowGTransformer()),  
    ('scaler', StandardScaler()),  
    ('regressor', Ridge())  
])
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
2562,Giannis Antetokounmpo,0.956,0.620201
4809,Luka Dončić,0.996,0.543658
6545,Shai Gilgeous-Alexander,1.000,0.527988
5635,Nikola Jokić,1.000,0.448247
4702,LeBron James,0.331,0.431208
399,Anthony Davis,0.465,0.428940
4317,Kevin Durant,0.554,0.414577
3432,Jayson Tatum,0.863,0.359249
7365,Victor Wembanyama,0.022,0.339653
1706,DeMar DeRozan,0.002,0.334225


In [45]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.8624417751148972

In [46]:
from sklearn.linear_model import Lasso
pipeline = Pipeline(steps=[
    ('replace_low_g', ReplaceLowGTransformer()),  
    ('scaler', StandardScaler()),  
    ('regressor', Lasso(alpha=0.5))  
])
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
1,A.J. Green,0.0,0.017907
4923,Malik Williams,0.0,0.017907
4872,Malachi Flynn,0.0,0.017907
4877,Malaki Branham,0.0,0.017907
4885,Malcolm Brogdon,0.0,0.017907
4886,Malcolm Cazalon,0.0,0.017907
4910,Malik Beasley,0.0,0.017907
4920,Malik Monk,0.0,0.017907
4927,Mamadi Diakite,0.0,0.017907
4833,Luke Kennard,0.0,0.017907


In [47]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.0

## SVC

In [48]:
from sklearn.svm import SVR
pipeline = Pipeline(steps=[
    ('replace_low_g', ReplaceLowGTransformer()),  
    ('scaler', StandardScaler()),  
    ('regressor', SVR(kernel='rbf', C=100, gamma=0.1))  
])
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
3432,Jayson Tatum,0.863,0.755784
5635,Nikola Jokić,1.000,0.739628
2562,Giannis Antetokounmpo,0.956,0.702222
4809,Luka Dončić,0.996,0.668908
6545,Shai Gilgeous-Alexander,1.000,0.660155
4702,LeBron James,0.331,0.652705
7288,Tyrese Maxey,0.032,0.604330
4317,Kevin Durant,0.554,0.541839
3071,Jalen Brunson,0.743,0.514480
403,Anthony Edwards,0.576,0.514043


In [49]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.9251261997013829

In [50]:
# pipeline = Pipeline(steps=[
#     ('replace_low_g', ReplaceLowGTransformer()),  
#     ('scaler', StandardScaler()),  
#     ('regressor', SVR(kernel='poly', degree =3,C=10, gamma=0.1))  
# ])
# pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

# predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
# predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
# wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
# wyniki_all_nba

In [51]:
# true_scores = np.array(wyniki_all_nba['Share'])
# predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
# ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
# ndcg

In [52]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingRegressor
# from sklearn.gaussian_process import GaussianProcessRegressor
pipeline = Pipeline([
    ('replace_low_g', ReplaceLowGTransformer()),
    ('standard_scaler', StandardScaler()),
    ('regressor', VotingRegressor([
        ('GBR', GradientBoostingRegressor()),
        # ('Gaus',GaussianProcessRegressor()),
        ('SVR', SVR(C=1000, degree=1, gamma='auto'))
        
    ]))
])
pipeline.fit(train_all_nba[p_all_nba],train_all_nba["Share"])

predykcja_all_nba = pipeline.predict(test_all_nba[p_all_nba])
predykcja_all_nba= pd.DataFrame(predykcja_all_nba,columns =["Wynik all nba"],index=test_all_nba.index)
wyniki_all_nba = pd.concat([test_all_nba[["Player","Share"]],predykcja_all_nba ] ,axis=1).sort_values("Wynik all nba",ascending = False).head(20)
wyniki_all_nba

,Player,Share,Wynik all nba
6545,Shai Gilgeous-Alexander,1.000,1.063432
4809,Luka Dončić,0.996,1.006845
2562,Giannis Antetokounmpo,0.956,0.848047
5635,Nikola Jokić,1.000,0.810880
4702,LeBron James,0.331,0.767975
4317,Kevin Durant,0.554,0.763567
3432,Jayson Tatum,0.863,0.690316
403,Anthony Edwards,0.576,0.595910
1892,Devin Booker,0.141,0.585692
6698,Stephen Curry,0.236,0.556392


In [53]:
true_scores = np.array(wyniki_all_nba['Share'])
predicted_scores = np.array(wyniki_all_nba['Wynik all nba'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=15)
ndcg

0.9586130428351991


# Szukanie modelu dla all-rookie


## LinearReg

In [54]:
pipeline = Pipeline(steps=[
    # ('scaler', StandardScaler()),  # Optional: Scale features
    ('regressor', LinearRegression())  # Linear Regression model
])

pipeline.fit(train_all_rookie[p_all_rookie],train_all_rookie["Share"])

Pipeline(steps=[('regressor', LinearRegression())])

In [55]:
predykcja_rookie = pipeline.predict(test_all_rookie[p_all_rookie])
predykcja_rookie= pd.DataFrame(predykcja_rookie,columns =["Wynik rookie"],index=test_all_rookie.index)
wyniki_all_rookie = pd.concat([test_all_rookie[["Player","Share"]],predykcja_rookie ] ,axis=1).sort_values("Wynik rookie",ascending = False).head(20)
wyniki_all_rookie




,Player,Share,Wynik rookie
1298,Victor Wembanyama,1.000,1.453992
183,Chet Holmgren,1.000,1.139633
122,Brandon Miller,0.985,0.814428
793,Keyonte George,0.389,0.576197
1129,Scoot Henderson,0.167,0.519110
537,Jaime Jaquez Jr.,0.949,0.516485
43,Amen Thompson,0.505,0.414132
114,Brandin Podziemski,0.813,0.408643
420,GG Jackson II,0.217,0.394557
1235,Trayce Jackson-Davis,0.212,0.352540


In [56]:
spearman_corr, _ = spearmanr(wyniki_all_rookie['Share'], wyniki_all_rookie['Wynik rookie'])
spearman_corr

0.7807103598163856

In [57]:
true_scores = np.array(wyniki_all_rookie['Share'])
predicted_scores = np.array(wyniki_all_rookie['Wynik rookie'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=10)
ndcg

0.922861596612585

## Gradient

In [58]:
# from sklearn.ensemble import GradientBoostingRegressor
pipeline = Pipeline(steps=[
    # ('replace_low_g', ReplaceLowGTransformer()),  
    ('scaler', StandardScaler()),  
    ('regressor', GradientBoostingRegressor())  
])

In [59]:

pipeline.fit(train_all_rookie[p_all_rookie],train_all_rookie["Share"])
predykcja_rookie = pipeline.predict(test_all_rookie[p_all_rookie])
predykcja_rookie= pd.DataFrame(predykcja_rookie,columns =["Wynik rookie"],index=test_all_rookie.index)
wyniki_all_rookie = pd.concat([test_all_rookie[["Player","Share"]],predykcja_rookie ] ,axis=1).sort_values("Wynik rookie",ascending = False).head(20)
wyniki_all_rookie

,Player,Share,Wynik rookie
1298,Victor Wembanyama,1.000,1.035613
183,Chet Holmgren,1.000,0.996967
122,Brandon Miller,0.985,0.901404
1129,Scoot Henderson,0.167,0.544439
43,Amen Thompson,0.505,0.520076
114,Brandin Podziemski,0.813,0.506419
537,Jaime Jaquez Jr.,0.949,0.474433
793,Keyonte George,0.389,0.414796
420,GG Jackson II,0.217,0.370054
81,Ausar Thompson,0.177,0.326159


In [60]:
spearman_corr, _ = spearmanr(wyniki_all_rookie['Share'], wyniki_all_rookie['Wynik rookie'])
spearman_corr

0.760256412344997

In [61]:
true_scores = np.array(wyniki_all_rookie['Share'])
predicted_scores = np.array(wyniki_all_rookie['Wynik rookie'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=10)
ndcg

0.9202529993252835

## SVC

In [62]:
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),  
    ('regressor', SVR(kernel='rbf', C=100, gamma=0.1))  
])

pipeline.fit(train_all_rookie[p_all_rookie],train_all_rookie["Share"])
predykcja_rookie = pipeline.predict(test_all_rookie[p_all_rookie])
predykcja_rookie= pd.DataFrame(predykcja_rookie,columns =["Wynik rookie"],index=test_all_rookie.index)
wyniki_all_rookie = pd.concat([test_all_rookie[["Player","Share"]],predykcja_rookie ] ,axis=1).sort_values("Wynik rookie",ascending = False).head(20)
wyniki_all_rookie

,Player,Share,Wynik rookie
122,Brandon Miller,0.985,0.898333
183,Chet Holmgren,1.000,0.529665
114,Brandin Podziemski,0.813,0.515940
537,Jaime Jaquez Jr.,0.949,0.512060
793,Keyonte George,0.389,0.492343
43,Amen Thompson,0.505,0.476575
1298,Victor Wembanyama,1.000,0.423484
1129,Scoot Henderson,0.167,0.385424
364,Duop Reath,0.010,0.372256
1235,Trayce Jackson-Davis,0.212,0.349056


In [63]:
spearman_corr, _ = spearmanr(wyniki_all_rookie['Share'], wyniki_all_rookie['Wynik rookie'])
spearman_corr

0.7930525977513216

In [64]:
true_scores = np.array(wyniki_all_rookie['Share'])
predicted_scores = np.array(wyniki_all_rookie['Wynik rookie'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=10)
ndcg

0.9134199360396688

##  RandomForest


In [65]:
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),  # Optional: Sca1le features
    ('regressor', RandomForestRegressor(n_estimators=100, max_depth=20))  # Linear Regression model
])

pipeline.fit(train_all_rookie[p_all_rookie],train_all_rookie["Share"])
predykcja_rookie = pipeline.predict(test_all_rookie[p_all_rookie])
predykcja_rookie= pd.DataFrame(predykcja_rookie,columns =["Wynik rookie"],index=test_all_rookie.index)
wyniki_all_rookie = pd.concat([test_all_rookie[["Player","Share"]],predykcja_rookie ] ,axis=1).sort_values("Wynik rookie",ascending = False).head(20)
wyniki_all_rookie

,Player,Share,Wynik rookie
1298,Victor Wembanyama,1.000,0.962620
122,Brandon Miller,0.985,0.928490
183,Chet Holmgren,1.000,0.915220
1129,Scoot Henderson,0.167,0.622930
537,Jaime Jaquez Jr.,0.949,0.550770
793,Keyonte George,0.389,0.489840
114,Brandin Podziemski,0.813,0.422650
43,Amen Thompson,0.505,0.420680
420,GG Jackson II,0.217,0.349050
81,Ausar Thompson,0.177,0.341820


In [66]:
spearman_corr, _ = spearmanr(wyniki_all_rookie['Share'], wyniki_all_rookie['Wynik rookie'])
spearman_corr

0.8364753855175562

In [67]:
true_scores = np.array(wyniki_all_rookie['Share'])
predicted_scores = np.array(wyniki_all_rookie['Wynik rookie'])
ndcg = ndcg_score([true_scores], [predicted_scores], k=10)
ndcg

0.9231047368963868